### reference
https://partner.cloudskillsboost.google/course_templates/948/video/485086

In [ ]:
!pip3 install --upgrade --user google-cloud-aiplatform umap-learn tqdm pypdf

In [ ]:
from vertexai.preview.generative_models import GenerativeModel
from vertexai.language_models import TextEmbeddingModel
text_embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")
model = GenerativeModel('gemini-pro')

In [44]:
import pandas as pd

data=pd.read_csv('testdata.csv',encoding='latin1')
data

,title,content
0,intelia ethos,\t\n#delightthecustomer\n\nOur customers' succ...
1,Migrating Data from Legacy Enterprise Data War...,Migrating data from a legacy on-prem data ware...
2,Data is at our core. It's what we're passionat...,"intelia was founded in 2018, with a desire to ..."
3,intelia has a unique technology partner ecosys...,In an increasingly fast-paced and competitive ...
4,? Celebrating Excellence at intelia! ?,"ast night, we came together as hashtag#onetrib..."
5,Direnc Uysal's linked-in post,Amidst the excitement surrounding Generative A...
6,intelia's post on 22rd december,"As 2024 comes to a close, the intelia hashtag#..."


In [45]:
from google.cloud import storage
import tempfile, shutil
import time
from datetime import datetime
import json
from google.cloud import bigquery 
import os 
import gcsfs
import pandas as pd

def upload_file(request_file : tempfile,dest_bucket_name:str =None,request_file_folder: str =None,request_file_prefix: str =None, version: int=0, request_file_post_fix : str=""):

    """upload file into gcs
   
        Args:
            tempfile request_file: request file
            str dest_bucket_name:  name of destination bucket
            str request_file_folder: name of the destination folder name to write files to
            list request_file_prefix: prefix of request file name
          
    """

    temp=request_file
    client = storage.Client()
    # Extract name to the temp file
    temp_file = "".join([str(temp.name)])
    # Uploading the temp image file to the bucket
    dest_filename = f"{request_file_folder}/"+request_file_prefix+'_'+request_file_post_fix+'_'+str(version)+".json" 
    dest_bucket = client.get_bucket(dest_bucket_name)
    dest_blob = dest_bucket.blob(dest_filename)
    dest_blob.upload_from_filename(temp_file)      

In [49]:
data

,title,content
0,intelia ethos,\t\n#delightthecustomer\n\nOur customers' succ...
1,Migrating Data from Legacy Enterprise Data War...,Migrating data from a legacy on-prem data ware...
2,Data is at our core. It's what we're passionat...,"intelia was founded in 2018, with a desire to ..."
3,intelia has a unique technology partner ecosys...,In an increasingly fast-paced and competitive ...
4,? Celebrating Excellence at intelia! ?,"ast night, we came together as hashtag#onetrib..."
5,Direnc Uysal's linked-in post,Amidst the excitement surrounding Generative A...
6,intelia's post on 22rd december,"As 2024 comes to a close, the intelia hashtag#..."


In [52]:
request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
rf= open(request_file.name, "a") 

for idx,row in data.iterrows():
    data_items=[ json.dumps( {  "_id": idx,
                                "title":row['title'],
                                 "text":row['content']                                          
                               }
                               )  +"\n"
                ]

    rf.writelines(data_items)
    rf.flush()
upload_file(rf,dest_bucket_name='vlt_search_and_contet_output',request_file_folder='finetunning',request_file_prefix='fine_tuning_intelia_sample', version=0, request_file_post_fix ='0')


In [ ]:
def generate_queries(model, title, text):

    prompt = (f'You are a human being who is tasked with extracting some questions from a given text.\n'
    f'Think throughly when reading this text and see what sort of questions may people ask from the content from a data consultancy called Intelia.\n'
    f'Suggest 1 to 5 short questions that are important to ask when people read this text or want to know more about Intelia.\n'    
    f'Output each question on a separate line divided by a newline.\n'
    f'Here is the text:\n'
    f'"title": {title}\n'
    f'"content":{text}')
    
    responses = model.generate_content(prompt, stream=False)
    return responses.text.split('\n')

generated_queries = generate_queries(model)
for query in generated_queries:
    print(query)

In [ ]:
generated_queries=[]
request_file = tempfile.NamedTemporaryFile(suffix=".json", delete=True) 
rf= open(request_file.name, "a") 

for idx,row in data.iterrows():
    queries=generate_queries(model, row['title'], row['content'])
    generated_queries.append()


for idx,row in data.iterrows():
    data_items=[ json.dumps( {  "_id": idx,
                                "title":row['title'],
                                 "text":row['content']                                          
                               }
                               )  +"\n"
                ]

    rf.writelines(data_items)
    rf.flush()
upload_file(rf,dest_bucket_name='vlt_search_and_contet_output',request_file_folder='finetunning',request_file_prefix='fine_tuning_intelia_sample', version=0, request_file_post_fix ='0')

    
    

In [ ]:
def evaluate_results(query, statement, model):

    prompt = (f'You are a helpful expert financial research assistant.\n'
    f'You help users analyze financial statements to better understand companies.\n'
    f'For the given query, evaluate whether the following satement is relevant.\n'
    f'Output only \'yes\' or \'no\'.\n'
    f'Question: {query}, Statement: {statement}.')
    
    responses = model.generate_content(prompt, stream=False)
    if responses.text == 'yes':
        return 1
    return -1

retrieved_embeddings = results['embeddings']
query_embeddings = embedding_function(generated_queries)

In [ ]:
adapter_query_embeddings = []
adapter_doc_embeddings = []
adapter_labels = []

for q, query in enumerate(tqdm(generated_queries)):
    for d, document in enumerate(retrieved_documents[q]):
        adapter_query_embeddings.append(query_embeddings[q])
        adapter_doc_embeddings.append(retrieved_embeddings[q][d])
        adapter_labels.append(evaluate_results(query, document, model))

len(adapter_labels)

In [ ]:
!pip install torch

import torch

In [ ]:
adapter_query_embeddings = torch.Tensor(np.array(adapter_query_embeddings))
adapter_doc_embeddings = torch.Tensor(np.array(adapter_doc_embeddings))
adapter_labels = torch.Tensor(np.expand_dims(np.array(adapter_labels),1))

dataset = torch.utils.data.TensorDataset(adapter_query_embeddings, adapter_doc_embeddings, adapter_labels)

In [ ]:
def model(query_embedding, document_embedding, adaptor_matrix):
    updated_query_embedding = torch.matmul(adaptor_matrix, query_embedding)
    return torch.cosine_similarity(updated_query_embedding, document_embedding, dim=0)

In [ ]:
def mse_loss(query_embedding, document_embedding, adaptor_matrix, label):
    return torch.nn.MSELoss()(model(query_embedding, document_embedding, adaptor_matrix), label)

In [ ]:
mat_size = len(adapter_query_embeddings[0])
adapter_matrix = torch.randn(mat_size, mat_size, requires_grad=True)

In [ ]:
min_loss = float('inf')
best_matrix = None

for epoch in tqdm(range(100)):
    for query_embedding, document_embedding, label in dataset:
        loss = mse_loss(query_embedding, document_embedding, adapter_matrix, label)

        if loss < min_loss:
            min_loss = loss
            best_matrix = adapter_matrix.clone().detach().numpy()

        loss.backward()
        with torch.no_grad():
            adapter_matrix -= 0.01 * adapter_matrix.grad
            adapter_matrix.grad.zero_()       

In [ ]:
print(f"Best loss: {min_loss.detach().numpy()}")

In [ ]:
test_vector = torch.ones((mat_size,1))
scaled_vector = np.matmul(best_matrix, test_vector).numpy()

In [ ]:
import matplotlib.pyplot as plt
plt.bar(range(len(scaled_vector)), scaled_vector.flatten())
plt.show()

In [ ]:
query_embeddings = embedding_function(generated_queries)
adapted_query_embeddings = np.matmul(best_matrix, np.array(query_embeddings).T).T

projected_query_embeddings = project_embeddings(query_embeddings, umap_transform)
projected_adapted_query_embeddings = project_embeddings(adapted_query_embeddings, umap_transform)

In [ ]:
# Plot the projected query and retrieved documents in the embedding space
plt.figure()
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10, color='gray')
plt.scatter(projected_query_embeddings[:, 0], projected_query_embeddings[:, 1], s=150, marker='X', color='r', label="original")
plt.scatter(projected_adapted_query_embeddings[:, 0], projected_adapted_query_embeddings[:, 1], s=150, marker='X', color='green', label="adapted")

plt.gca().set_aspect('equal', 'datalim')
plt.title("Adapted Queries")
plt.axis('off')
plt.legend()